In [1]:
! pip install sentencepiece

In [3]:
import torch
import json
from torch.utils.data import Dataset, DataLoader
#from transformers import DebertaV2Tokenizer as tkz
from transformers import DebertaV2ForMultipleChoice as mpc
from transformers import DebertaTokenizer as tkz
#from transformers import DebertaForSequenceClassification as mpc
import torch.optim as optim

In [4]:
# Reading the train dataset back
with open('/kaggle/input/cf-dataset/train_dataset_final.json', 'r') as train_file:
    train_dataset_back = json.load(train_file)

# Reading the test dataset back
with open('/kaggle/input/cf-dataset/test_dataset_final.json', 'r') as test_file:
    test_dataset_back = json.load(test_file)

# Display the sizes of the two datasets
print(f"Size of the train dataset: {len(train_dataset_back)}")
print(f"Size of the test dataset: {len(test_dataset_back)}")

Size of the train dataset: 1338
Size of the test dataset: 599


In [5]:
class TTDataset(Dataset):
    def __init__(self, copa_data, tokenizer):
        self.copa_data = copa_data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.copa_data)

    def __getitem__(self, idx):
        data = self.copa_data[idx]
        premise = data['premise']
        hypotheses = [data['initial'], data['counterfactual']]
        label = data['label']

        # Tokenize
        encoded_input = self.tokenizer([premise] * 2, hypotheses, padding='max_length', truncation=True, return_tensors='pt')
        encoded_input['labels'] = torch.tensor(label)

        return encoded_input


In [6]:
model_base = 'microsoft/deberta-base'

# Load tokenizer and model
tokenizer = tkz.from_pretrained(model_base)
model = mpc.from_pretrained(model_base)

# Prepare the DataLoader
TT_dataset = TTDataset(train_dataset_back, tokenizer)
dataloader = DataLoader(TT_dataset, batch_size=4, shuffle=True)

# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

You are using a model of type deberta to instantiate a model of type deberta-v2. This is not supported for all configurations of models and can yield errors.
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'deberta.encoder.layer.0.attention.self.key_proj.bias', 'deberta.encoder.layer.0.attention.self.key_proj.weight', 'deberta.encoder.layer.0.attention.self.pos_key_proj.bias', 'deberta.encoder.layer.0.attention.self.pos_key_proj.weight', 'deberta.encoder.layer.0.atten

In [7]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 33.7 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# Training Loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [1]:
# Training Loop
c=0
for epoch in range(3):  # Number of epochs
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(0)
        attention_mask = batch['attention_mask'].squeeze(0)
        labels = batch['labels'].squeeze(0)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        c+=1

        print(f"count:{c},Epoch: {epoch}, Loss: {loss.item()}")

print('Training done')

NameError: name 'model' is not defined

In [ ]:
#tokenizer = tkz.from_pretrained(model_base)
validation_dataset = TTDataset(test_dataset_back, tokenizer)
validation_dataloader = DataLoader(validation_dataset, batch_size=8)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.eval()
correct_predictions = 0
total_predictions = 0

accuracy, f1, precision, recall = [], [], [], []
batches = 0

def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    return [accuracy, f1, precision, recall]

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']  # ground truth

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)  # predictions

        calculated_metrics = compute_metrics(labels, predictions)
        accuracy.append(calculated_metrics[0])
        f1.append(calculated_metrics[1])
        precision.append(calculated_metrics[2])
        recall.append(calculated_metrics[3])
        batches += 1

        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")

print(f"Accuracy: {sum(accuracy)/batches:.4f}")
print(f"F1: {sum(f1)/batches:.4f}")
print(f"Precision: {sum(precision)/batches:.4f}")
print(f"Recall: {sum(recall)/batches:.4f}")

print(f'Number of batches: {batches}')

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")

In [ ]:
# Specify the path where you want to save the model
model_path = '/content/model_DeBERTA.pth'

# Save the entire model
torch.save(model, model_path)


In [ ]:
model_sd_path = '/content/model_DeBERTA_sd.pth'
torch.save(model.state_dict(), model_sd_path)

In [ ]:
model = mpc.from_pretrained(model_base)
model.load_state_dict(torch.load(model_sd_path))

In [ ]:
# Define split sizes for a 75-25 split
val_dataset_size = int(0.10 * len(test_dataset_back))

val_data = test_dataset_back[:val_dataset_size]
len(val_data)

In [ ]:
tokenizer = tkz.from_pretrained(model_base)
val_dataset = TTDataset(val_data, tokenizer)
val_dataloader = DataLoader(val_dataset, batch_size=8)

In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")

# Neuro Symbolic test

In [ ]:
class COPADataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataframe = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        try:
            # Extracting data from the dataframe
            data = self.dataframe.iloc[idx]
            premise = data['premise']
            hypothesis1 = data['hypothesis1']
            hypothesis2 = data['hypothesis2']
            label = data['label']

            # Concatenating the premise and hypotheses with their respective relations
            premise_with_relations = premise + " " + data['premise_relations']
            hypothesis1_with_relations = hypothesis1 + " " + data['hypothesis1_relations']
            hypothesis2_with_relations = hypothesis2 + " " + data['hypothesis2_relations']

            # Tokenize
            encoded_input = self.tokenizer(
                [premise_with_relations] * 2,
                [hypothesis1_with_relations, hypothesis2_with_relations],
                padding='max_length',
                truncation=True,
                max_length=512,  # Or another suitable max length
                return_tensors='pt'
            )
            encoded_input['labels'] = torch.tensor(label)

            return encoded_input

        except KeyError as e:
            print(f"KeyError encountered at index {idx}: {e}")
            raise


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertForMultipleChoice
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

processed_data = pd.read_csv('/content/processed_train_data.csv')
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')

# Prepare the DataLoader
# Convert your dataframe to the updated dataset
train_dataset = COPADataset(processed_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)


# Optimizer and Loss Function
optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training Loop
model.train()
for epoch in range(3):  # Number of epochs
    for batch in train_dataloader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [ ]:
val_data = pd.read_csv('/content/processed_validation_data.csv')
# Load tokenizer and model

validation_dataset = COPADataset(val_data, tokenizer)
validation_dataloader = DataLoader(validation_dataset, batch_size=8)


In [ ]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in validation_dataloader:
        input_ids = batch['input_ids'].squeeze(1)
        attention_mask = batch['attention_mask'].squeeze(1)
        labels = batch['labels']

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)
        correct_predictions += (predictions == labels).sum().item()
        total_predictions += labels.size(0)

overall_accuracy = correct_predictions / total_predictions
print(f"Overall Accuracy: {overall_accuracy:.4f}")
